In [1]:
#!pip install selenium
#!pip install webdriver-manager


In [2]:
#import selenium
#print(selenium.__version__)


In [ ]:
import re
import nltk
import string
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
import json
import time

# Ensure necessary NLTK resources are downloaded
# nltk.download('punkt')
# nltk.download('wordnet')

# Initialize lemmatizer and stemmer
lemmer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Functions for tokenization, lemmatization, and stemming
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]

def StemTokens(tokens):
    return [stemmer.stem(token) for token in tokens]

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

# Read responses from a JSON file
def load_responses(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except Exception as e:
        print(f"Error loading file: {e}")
        return {}

# Detect if user input is a Python or Java error
def detect_error(user_input):
    # Common Python error patterns
    python_error_pattern = (
        r"Traceback \(most recent call last\):|"
        r"NameError|TypeError|ValueError|ImportError|IndentationError|"
        r"KeyError|IndexError|AttributeError|ModuleNotFoundError|SyntaxError"
    )
    
    # Common Java error patterns
    java_error_pattern = (
        r"Exception in thread|NullPointerException|ArrayIndexOutOfBoundsException|"
        r"ClassNotFoundException|IllegalArgumentException|NumberFormatException|"
        r"FileNotFoundException|NoClassDefFoundError|ArithmeticException|"
        r"UnsupportedOperationException"
    )
    
    if re.search(python_error_pattern, user_input, re.IGNORECASE):
        return "python"
    elif re.search(java_error_pattern, user_input, re.IGNORECASE):
        return "java"
    return None

# Open Stack Overflow and search for the error
def search_stack_overflow(error_type, user_input, error_responses):
    # Find the appropriate Stack Overflow URL for the error type
    if error_type in error_responses:
        base_url = error_responses[error_type]
    else:
        return "I'm sorry, I couldn't find a Stack Overflow URL for this error type."

    # Use Selenium to automate the search
    try:
        # Initialize Firefox with GeckoDriverManager
        service = Service(GeckoDriverManager().install())
        driver = webdriver.Firefox(service=service)
        
        search_url = f"{base_url}+{user_input.replace(' ', '+')}"
        driver.get(search_url)

        # Wait for page to load and retrieve the first result
        wait = WebDriverWait(driver, 10)
        results = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.result-link > a')))
        
        if results:
            first_result_url = results[0].get_attribute('href')
            driver.quit()
            return f"Here is a suggested solution: {first_result_url}"
        else:
            driver.quit()
            return "No solutions were found on Stack Overflow for your error."
    except Exception as e:
        return f"An error occurred while accessing Stack Overflow: {str(e)}"

# Provide solutions for detected errors
def get_error_solution(error_type, user_input, error_responses):
    return search_stack_overflow(error_type, user_input, error_responses)

# Chatbot response logic
def response(user_response, error_responses):
    error_type = detect_error(user_response)
    if error_type:  # If an error is detected
        return get_error_solution(error_type, user_response, error_responses)
    return "Sorry, I didn't recognize the error. Could you provide more details?"

# Main chat loop
responses_data = load_responses('responses.json')  # Load responses from JSON file

# Check if responses_data is loaded properly
if not responses_data:
    print("Error: Responses data not loaded correctly.")
    exit()

error_responses = responses_data.get("error_responses", {})  # Extract the error responses

# If error_responses is empty, exit the program
#if not error_responses:
    #print("Error: No error responses found in the JSON file.")
    #exit()

print("CoxBot: Hello! I can help you with Python or Java errors. Type 'bye' to exit.")

while True:
    user_response = input("You: ").lower()
    if user_response == 'bye':
        print("CoxBot: Goodbye! Have a nice day!")
        break
    elif user_response in ['thanks', 'thank you']:
        print("CoxBot: You are welcome!")
    else:
        print("CoxBot: ", end="")
        print(response(user_response, error_responses))


CoxBot: Hello! I can help you with Python or Java errors. Type 'bye' to exit.


You:  Hi


CoxBot: Sorry, I didn't recognize the error. Could you provide more details?
